# Python for analysis: Scikit-HEP tutorial

**June 20, 2025 at HSF/IRIS-HEP Software Basics Training at CERN**

based on [Scikit-HEP Tutorial](https://hsf-training.github.io/hsf-training-scikit-hep-webpage/index.html)

<br><br><br><br><br>

## Z peak in PyROOT

Let's start with the most straightforward way to analyze HEP data in Python: "for" loops in PyROOT.

In [ ]:
import time
import numpy as np
import ROOT

canvas = ROOT.TCanvas()

In [ ]:
rootfile = ROOT.TFile.Open("data/HiggsZZ4mu.root")
roottree = rootfile.Get("Events")

In [ ]:
starttime = time.time()

roothist = ROOT.TH1D("roothist", "mass", 120, 0, 120)

for index, event in enumerate(roottree):
    # cuts are "if" statements
    if event.nMuon >= 2 and event.Muon_charge[0] + event.Muon_charge[1] == 0:
        mu1_pt = event.Muon_pt[0]
        mu2_pt = event.Muon_pt[1]
        mu1_eta = event.Muon_eta[0]
        mu2_eta = event.Muon_eta[1]
        mu1_phi = event.Muon_phi[0]
        mu2_phi = event.Muon_phi[1]

        # histograms are filled in the loop
        roothist.Fill(
            np.sqrt(2*mu1_pt*mu2_pt*(np.cosh(mu1_eta - mu2_eta) - np.cos(mu1_phi - mu2_phi)))
        )

pyroot_time = time.time() - starttime
print(f"total time: {pyroot_time} sec")

In [ ]:
roothist.Draw()
canvas.Draw()

<br><br><br><br><br>

## Z peak in C++ ROOT

It works, but 3 seconds is a long time to wait for 299683 dimuons.

The traditional way to speed things up is to translate it into C++.

In [ ]:
ROOT.gInterpreter.Declare('''
void compute(TH1D& roothist, TTree& roottree) {
    UInt_t nMuon;
    float Muon_pt[50];
    float Muon_eta[50];
    float Muon_phi[50];
    int32_t Muon_charge[50];

    roottree.SetBranchStatus("*", 0);
    roottree.SetBranchStatus("nMuon", 1);
    roottree.SetBranchStatus("Muon_pt", 1);
    roottree.SetBranchStatus("Muon_eta", 1);
    roottree.SetBranchStatus("Muon_phi", 1);
    roottree.SetBranchStatus("Muon_charge", 1);

    roottree.SetBranchAddress("nMuon", &nMuon);
    roottree.SetBranchAddress("Muon_pt", Muon_pt);
    roottree.SetBranchAddress("Muon_eta", Muon_eta);
    roottree.SetBranchAddress("Muon_phi", Muon_phi);
    roottree.SetBranchAddress("Muon_charge", Muon_charge);

    for (int index = 0; index < 100000; index++) {
        roottree.GetEntry(index);
        if (nMuon >= 2 && Muon_charge[0] + Muon_charge[1] == 0) {
            float mu1_pt = Muon_pt[0];
            float mu2_pt = Muon_pt[1];
            float mu1_eta = Muon_eta[0];
            float mu2_eta = Muon_eta[1];
            float mu1_phi = Muon_phi[0];
            float mu2_phi = Muon_phi[1];
            roothist.Fill(
                sqrt(2*mu1_pt*mu2_pt*(cosh(mu1_eta - mu2_eta) - cos(mu1_phi - mu2_phi)))
            );
        }
    }
}
''')

ROOT lets you compile a C++ function and run it through PyROOT.

The important thing here is that the _loop over events_ is in the compiled code.

In [ ]:
starttime = time.time()

roothist2 = ROOT.TH1D("roothist2", "mass", 120, 0, 120)

ROOT.compute(roothist2, roottree)

cpproot_time = time.time() - starttime
print(f"total time: {cpproot_time} sec")

In [ ]:
pyroot_time / cpproot_time

In [ ]:
roothist2.Draw()
canvas.Draw()

<br><br><br><br><br>

## Z peak in RDataFrame

RDataFrame is the modern way to build workflows over HEP data in ROOT.

Each node in this pipeline is compiled in C++. The pipeline itself can be built in Python.

<img src="img/rdataframe-flow.svg" style="width: 800px">

In [ ]:
df = ROOT.RDataFrame("Events", "data/HiggsZZ4mu.root")

# Each node is connected to the previous, in a chain (which can split and recombine).
df_2mu = df.Filter("nMuon >= 2")
df_os = df_2mu.Filter("Muon_charge[0] + Muon_charge[1] == 0")

# This node is a big C++ block.
df_mass = df_os.Define("Dimuon_mass", '''
float mu1_pt = Muon_pt[0];
float mu2_pt = Muon_pt[1];
float mu1_eta = Muon_eta[0];
float mu2_eta = Muon_eta[1];
float mu1_phi = Muon_phi[0];
float mu2_phi = Muon_phi[1];
return sqrt(2*mu1_pt*mu2_pt*(cosh(mu1_eta - mu2_eta) - cos(mu1_phi - mu2_phi)));
''')

roothist3 = df_mass.Histo1D(("h3", "mass", 120, 0, 120), "Dimuon_mass")

The calculation doesn't actually start until you attempt to look at a result (convert to NumPy or draw a plot).

In [ ]:
starttime = time.time()

# This one is an endpoint (action).
array = df_mass.AsNumpy(["Dimuon_mass"])

rdfroot_time = time.time() - starttime
print(f"total time: {rdfroot_time} sec")

In [ ]:
pyroot_time / rdfroot_time

In [ ]:
array

In [ ]:
roothist3.Draw()
canvas.Draw()

<br><br><br><br><br>

## Uproot and Awkward Array

Uproot's approach is different: each Python command operates on a whole array at a time (like NumPy).

Loops over all events happen in compiled code, but you don't write that code—you combine primitives to build a workflow.

In [ ]:
import awkward as ak
import uproot
import hist

Open a file and look at its contents.

In [ ]:
events = uproot.open("data/HiggsZZ4mu.root:Events")
events.show()

Extract some branches as an "array of records" (renaming the fields).

In [ ]:
muons = events.arrays(
    ["pt", "eta", "phi", "charge"],
    aliases={"pt": "Muon_pt", "eta": "Muon_eta", "phi": "Muon_phi", "charge": "Muon_charge"}
)
muons

In [ ]:
muons.fields

A cut is an array of booleans, which we can construct as a formula.

In [ ]:
cut = (ak.num(muons.charge) >= 2) & (ak.sum(muons.charge[:, :2], axis=1) == 0)
cut

Applying a cut is a slice. In the same slice, we can pick the first (`0`) and second (`1`) muon in each event.

In [ ]:
mu1 = muons[cut, 0]
mu2 = muons[cut, 1]
mu1, mu2

Let's use the [hist](https://github.com/scikit-hep/hist#readme) library for histograms.

In [ ]:
h = hist.Hist.new.Reg(120, 0, 120, name="mass").Double()
h

In [ ]:
h.fill(np.sqrt(2*mu1.pt*mu2.pt*(np.cosh(mu1.eta - mu2.eta) - np.cos(mu1.phi - mu2.phi))))

And Matplotlib for plots.

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
h.plot();

<br><br><br><br><br>

Now let's do the whole thing in one cell and time it.

In [ ]:
starttime = time.time()

# read data
muons = events.arrays(
    ["pt", "eta", "phi", "charge"],
    aliases={"pt": "Muon_pt", "eta": "Muon_eta", "phi": "Muon_phi", "charge": "Muon_charge"},
    array_cache=None,   # no cheating!
)

# compute
cut = (ak.num(muons.charge) >= 2) & (ak.sum(muons.charge[:, :2], axis=1) == 0)
mu1 = muons[cut, 0]
mu2 = muons[cut, 1]
h = hist.Hist.new.Reg(120, 0, 120, name="mass").Double()
h.fill(np.sqrt(2*mu1.pt*mu2.pt*(np.cosh(mu1.eta - mu2.eta) - np.cos(mu1.phi - mu2.phi))))

uproot_time = time.time() - starttime
print(f"total time: {uproot_time} sec")

In [ ]:
pyroot_time / uproot_time

It's in the same ballpark as C++. It can be 1.5× to 2× slower, but it's much closer to C++ on a log plot than it is to Python "for" loops.

<br><br><br><br><br>

## The Pythonic HEP ecosystem

Uproot is not a framework, it _only_ does ROOT I/O. Awkward Array handles array manipulation, hist does histograms, etc.

It is part of this complete breakfast:

<img src="img/logo-parade.svg" width="800px">

<br><br><br><br><br>

## Uproot

Uproot is a Python package that reads and writes ROOT files and is only concerned with reading and writing (no analysis, no plotting, etc.). It interacts with NumPy, Awkward Array, and Pandas for computations, boost-histogram/hist for histogram manipulation and plotting, Vector for Lorentz vector functions and transformations, Coffea for scale-up, etc.

Uproot is implemented using only Python and Python libraries. It doesn’t have a compiled part or require a specific version of ROOT. (This means that if you do use ROOT for something other than I/O, your choice of ROOT version is not constrained by I/O.)

<img src="img/abstraction-layers.svg" width="800px">

As a consequence of being an independent implementation of ROOT I/O, Uproot might not be able to read/write certain data types. Which data types are not implemented is a moving target, as new ones are always being added. A good approach for reading data is to just try it and see if Uproot complains. For writing, see the lists of supported types in the Uproot documentation (blue boxes in the text).

<br><br><br><br><br>

The documentation is at [https://uproot.readthedocs.io/](https://uproot.readthedocs.io/).

<img src="img/uproot-documentation.png" width="700px"/>

<br><br><br><br><br>

## Reading data from a file

### Opening the file

To open a file for reading, pass the name of the file to [uproot.open](https://uproot.readthedocs.io/en/latest/uproot.reading.open.html). In scripts, it is good practice to use [Python’s with](https://realpython.com/python-with-statement/) statement to close the file when you’re done, but if you’re working interactively, you can use a direct assignment.

To access a remote file via HTTP or XRootD, use a `"http://..."`, `"https://..."`, or `"root://..."` URL. If the Python interface to XRootD is not installed, the error message will explain how to install it.

In [ ]:
import uproot

filename = "data/uproot-Event.root"

file = uproot.open(filename)

<br><br><br><br><br>

### Listing contents

This “file” object actually represents a directory, and the named objects in that directory are accessible with a dict-like interface. Thus, `keys`, `values`, and `items` return the key names and/or read the data. If you want to just list the objects without reading, use keys. (This is like ROOT’s `ls()`, except that you get a Python list.)

In [ ]:
file.keys()

Often, you want to know the type of each object as well, so [uproot.ReadOnlyDirectory](https://uproot.readthedocs.io/en/latest/uproot.reading.ReadOnlyDirectory.html) objects also have a classnames method, which returns a dict of object names to class names (without reading them).

In [ ]:
file.classnames()

<br><br><br><br><br>

### Reading a histogram

If you’re familiar with ROOT, `TH1F` would be recognizable as histograms and `TTree` would be recognizable as a dataset. To read one of the histograms, put its name in square brackets:

In [ ]:
h = file["hstat"]
h

Uproot doesn’t do any plotting or histogram manipulation, so the most useful methods of `h` begin with `“to”`: `to_boost` (boost-histogram), `to_hist` (hist), `to_numpy` (NumPy’s 2-tuple of contents and edges), `to_pyroot` (PyROOT), etc.

In [ ]:
h.to_hist().plot()

Uproot histograms also satisfy the [UHI plotting protocol](https://uhi.readthedocs.io/en/latest/plotting.html), so they have methods like `values` (bin contents), `variances` (errors squared), and `axes`.

In [ ]:
h.values()
h.variances()
list(h.axes[0])  # "x", "y", "z" or 0, 1, 2

<br><br><br>

**Three-minute exercise:** find the 2D histogram and plot it.

In [ ]:
histograms = uproot.open("data/HiggsZZ4mu_histograms.root")
histograms

In [ ]:
histograms.file

In Uproot, a directory is a dict-like object with subscripting (square brackets), [keys](https://uproot.readthedocs.io/en/latest/uproot.reading.ReadOnlyDirectory.html#keys), [values](https://uproot.readthedocs.io/en/latest/uproot.reading.ReadOnlyDirectory.html#values), [items](https://uproot.readthedocs.io/en/latest/uproot.reading.ReadOnlyDirectory.html#items), and [classnames](https://uproot.readthedocs.io/en/latest/uproot.reading.ReadOnlyDirectory.html#classnames) methods.

In [ ]:
histograms["Z"]

In [ ]:
histograms["Z"]["4mu"]

Try `recursive`, `filter_name`, and `filter_classname` arguments.

In [ ]:
histograms.keys()

Most histograms and graphs can be converted to types in other Python libraries.

Try the `to_hist()` method on this one.

In [ ]:
histograms["Z/all/massZto2muon"]

In [ ]:
histograms

## Solution (no peeking!)

In [ ]:
{key: value for key, value in histograms.classnames().items() if value == "TH2D"}

<br><br><br><br><br>

## Reading a TTree

A TTree represents a potentially large dataset. Getting it from the [uproot.ReadOnlyDirectory](https://uproot.readthedocs.io/en/latest/uproot.reading.ReadOnlyDirectory.html) only returns its TBranch names and types. The `show` method is a convenient way to list its contents:

In [ ]:
t = file["T"]
t.show()

Be aware that you can get the same information from `keys` (an [uproot.TTree](https://uproot.readthedocs.io/en/latest/uproot.behaviors.TTree.TTree.html) is dict-like), `typename`, and `interpretation`.

In [ ]:
t.keys()
t["event/fNtrack"]
t["event/fNtrack"].typename
t["event/fNtrack"].interpretation

(If an [uproot.TBranch](https://uproot.readthedocs.io/en/latest/uproot.behaviors.TBranch.TBranch.html) has no `interpretation`, it can’t be read by Uproot.)

The most direct way to read data from an [uproot.TBranch](https://uproot.readthedocs.io/en/latest/uproot.behaviors.TBranch.TBranch.html) is by calling its `array` method.

In [ ]:
t["event/fNtrack"].array()

We’ll consider other methods in the next lesson.

<br><br><br><br><br>

## Reading a… what is that?

<img src="img/can-uproot-read-it.svg" width="500px"/>


This file also contains an instance of type [TProcessID](https://root.cern.ch/doc/master/classTProcessID.html). These aren’t typically useful in data analysis, but Uproot manages to read it anyway because it follows certain conventions (it has “class streamers”). It’s presented as a generic object with an `all_members` property for its data members (through all superclasses).

In [ ]:
file["ProcessID0"]
file["ProcessID0"].all_members

Here’s a more useful example of that: a supernova search with the IceCube experiment has custom classes for its data, which Uproot reads and represents as objects with `all_members`.

In [ ]:
icecube = uproot.open("data/icecube-supernovae.root")
icecube.keys()

In [ ]:
icecube.classname_of("config/detector")

It is possible to read an `I3Eval_t` object because ROOT stores the "how to read" instructions in the file (called "streamers").

In [ ]:
icecube.file.show_streamers("I3Eval_t")

In [ ]:
icecube["config/detector"]

In [ ]:
icecube["config/detector"].all_members

In [ ]:
icecube["config/detector"].member("ChannelIDMap")

Unlike histograms, these objects have no methods to help you unpack the data; it's all in [members](https://uproot.readthedocs.io/en/latest/uproot.model.Model.html#members), [all_members](https://uproot.readthedocs.io/en/latest/uproot.model.Model.html#all-members), [has_member](https://uproot.readthedocs.io/en/latest/uproot.model.Model.html#has-member), and [member](https://uproot.readthedocs.io/en/latest/uproot.model.Model.html#member).

<br><br><br><br><br>

## Writing data to a file

Uproot’s ability to <i>write</i> data is more limited than its ability to <i>read data</i>, but some useful cases are possible.

<br><br><br><br>

### Opening files for writing

First of all, a file must be opened for writing, either by creating a completely new file or updating an existing one.

In [ ]:
output1 = uproot.recreate("completely-new-file.root")
output2 = uproot.update("existing-file.root")

(Uproot cannot write over a network; output files must be local.)

<br><br><br><br>

### Writing strings and histograms

These [uproot.WritableDirectory](https://uproot.readthedocs.io/en/latest/uproot.writing.writable.WritableDirectory.html) objects have a dict-like interface: you can put data in them by assigning to square brackets.

In [ ]:
output1["some_string"] = "This will be a TObjString."

output1["some_histogram"] = file["hstat"]

import numpy as np

output1["nested_directory/another_histogram"] = np.histogram(
    np.random.normal(0, 1, 1000000)
)

In ROOT, the name of an object is a property of the object, but in Uproot, it’s a key in the TDirectory that holds the object, so that’s why the name is on the left-hand side of the assignment, in square brackets. Only the data types listed in the blue box in [the documentation](https://uproot.readthedocs.io/en/latest/basic.html#writing-objects-to-a-file) are supported: mostly just histograms.

<br><br><br><br>

### Writing TTrees

TTrees are potentially large and might not fit in memory. Generally, you’ll need to write them in batches.

One way to do this is to assign the first batch and `extend` it with subsequent batches:

In [ ]:
import numpy as np

output1["tree1"] = {
    "x": np.random.randint(0, 10, 1_000_000),
    "y": np.random.normal(0, 1, 1_000_000),
}
output1["tree1"].extend(
    {"x": np.random.randint(0, 10, 1_000_000), "y": np.random.normal(0, 1, 1_000_000)}
)
output1["tree1"].extend(
    {"x": np.random.randint(0, 10, 1_000_000), "y": np.random.normal(0, 1, 1_000_000)}
)

another is to create an empty TTree with [uproot.WritableDirectory.mktree](https://uproot.readthedocs.io/en/latest/uproot.writing.writable.WritableDirectory.html#mktree), so that every write is an extension.

In [ ]:
output1.mktree("tree2", {"x": np.int32, "y": np.float64})
output1["tree2"].extend(
    {"x": np.random.randint(0, 10, 1_000_000), "y": np.random.normal(0, 1, 1_000_000)}
)
output1["tree2"].extend(
    {"x": np.random.randint(0, 10, 1_000_000), "y": np.random.normal(0, 1, 1_000_000)}
)
output1["tree2"].extend(
    {"x": np.random.randint(0, 10, 1_000_000), "y": np.random.normal(0, 1, 1_000_000)}
)

Performance tips are given in the next lesson, but in general, it pays to write few large batches, rather than many small batches.

The only data types that can be assigned or passed to `extend` are listed in the blue box in [this documentation](https://uproot.readthedocs.io/en/latest/basic.html#writing-ttrees-to-a-file). This includes jagged arrays (described in the lesson after next), but not more complex types.

<br><br><br><br>



### Reading and writing RNTuples

TTree has been the default format to store large datasets in ROOT files for decades. However, it has slowly become outdated and are not optimized for modern systems. This is where the RNTuple format comes in. It is a modern serialization format that is designed with modern systems in mind and is planned to replace TTree in the coming years. [Version 1.0.0.0](https://cds.cern.ch/record/2923186) is out and will be supported "forever".

<img src="img/RNTuple.png" width="800px"/>


RNTuples are much simpler than TTrees by design, and this time there is an official specification, which makes it much easier for third-party I/O packages like Uproot to support. Uproot already supports reading the full RNTuple specification, meaning that you can read any RNTuple you find in the wild. It also supports writing a large part of the specification, and intends to support as much as it makes sense for data analysis.

To ease the transition into RNTuples, we are designing the interface to match the one for TTrees as closely as possible. Let's look at a simple example for reading and writing RNTuples.

In [ ]:
import uproot

filename = ("data/ntpl001_staff_rntuple_v1-0-0-0.root")

file = uproot.open(filename)

This time, if we print the class names, we see that there is an RNTuple instead of a TTree.

In [ ]:
file.classnames()

Then to read the data from the RNTuple works in an analogous way.

In [ ]:
rntuple = file["Staff"]
data = rntuple.arrays()
data

Writing again works in a very similar way to TTrees. However, since TTrees are still the default format used in more places, writing something like `file[key] = data` will default to writing the data as a TTree. When we want to write an RNTuple, we need to specifically tell Uproot that we want to do so. For now, we need to use an Awkward Array (which will be covered in a later lesson) to specify the data, but the interface will be extended to match TTrees.

In [ ]:
import awkward as ak

data = ak.Array({"my_int_data": [1, 2, 3], "my_float_data": [1.0, 2.0, 3.0]})
more_data = ak.Array({"my_int_data": [4, 5, 6], "my_float_data": [4.0, 5.0, 6.0]})

output3 = uproot.recreate("new-file-with-rntuple.root")

rntuple = output3.mkrntuple("my_rntuple", data)
rntuple.extend(more_data)

For the rest of the tutorial we will stick to using TTrees since this is still the main data format that you'll encounter for now.

<br><br>



### 📌 Key Points
 * Uproot TDirectories and TTrees have a dict-like interface.
 * Uproot reading methods are primarily intended to get data into a more specialized library.
 * Uproot writing is more limited, but it can write histograms, TTrees, and RNTuples.


<br><br><br><br><br>

## TTree details

### ROOT file structure and terminology

A ROOT file ([ROOT TFile](https://root.cern.ch/doc/master/classTFile.html), [uproot.ReadOnlyFile](https://uproot.readthedocs.io/en/latest/uproot.reading.ReadOnlyFile.html)) is like a little filesystem containing nested directories ([ROOT TDirectory](https://root.cern.ch/doc/master/classTDirectory.html), [uproot.ReadOnlyDirectory](https://uproot.readthedocs.io/en/latest/uproot.reading.ReadOnlyDirectory.html)). In Uproot, nested directories are presented as nested dicts.

Any class instance ([ROOT TObject](https://root.cern.ch/doc/master/classTObject.html), [uproot.Model](https://uproot.readthedocs.io/en/latest/uproot.model.Model.html)) can be stored in a directory, including types such as histograms (e.g. [ROOT TH1](https://root.cern.ch/doc/master/classTH1.html), [uproot.behaviors.TH1.TH1](https://uproot.readthedocs.io/en/latest/uproot.behaviors.TH1.TH1.html)).

One of these classes, TTree ([ROOT TTree](https://root.cern.ch/doc/master/classTTree.html), [uproot.TTree](https://uproot.readthedocs.io/en/latest/uproot.behaviors.TTree.TTree.html)), is a gateway to large datasets. A TTree is roughly like a Pandas DataFrame in that it represents a table of data. The columns are called TBranches ([ROOT TBranch](https://root.cern.ch/doc/master/classTBranch.html), [uproot.TBranch](https://uproot.readthedocs.io/en/latest/uproot.behaviors.TBranch.TBranch.html)), which can be nested (unlike Pandas), and the data can have any C++ type (unlike Pandas, which can store any Python type).

A TTree is often too large to fit in memory, and sometimes (rarely) even a single TBranch is too large to fit in memory. Each TBranch is therefore broken down into TBaskets ([ROOT TBasket](https://root.cern/doc/master/classTBasket.html), [uproot.models.TBasket.Model_TBasket](https://uproot.readthedocs.io/en/latest/uproot.models.TBasket.Model_TBasket.html)), which are “batches” of data. (These are the same batches that each call to `extend` writes in the previous lesson.) TBaskets are the smallest unit that can be read from a TTree: if you want to read the first entry, you have to read the first TBasket.


<img src="img/terminology.svg" style="width: 800px">


As a data analyst, you’ll likely be concerned with TTrees and TBranches first-hand, but only TBaskets when efficiency issues come up. Files with large TBaskets might require a lot of memory to read; files with small TBaskets will be slower to read (in ROOT also, but especially in Uproot). Megabyte-sized TBaskets are usually ideal.

<br><br><br><br>

### Examples with a large TTree
[This file](https://opendata.web.cern.ch/record/12341) is 2.1 GB, hosted by CERN’s Open Data Portal.

In [ ]:
import uproot

file = uproot.open(
    ###"root://eospublic.cern.ch//eos/opendata/cms/derived-data/AOD2NanoAODOutreachTool/Run2012BC_DoubleMuParked_Muons.root"
    "data/HiggsZZ4mu.root"
)
file.classnames()

#### Why the ;74 and ;75?

You may have been wondering about the numbers after the semicolons. These are ROOT “cycle numbers,” which allow objects with the same name to be distinguishable. They’re used when an object needs to be overwritten as it grows without losing the last valid copy of that object, so that a ROOT file can be read even if the writing process failed partway through.

In this case, the last version of this TTree was number 75, and number 74 is the second-to-last.

If you don’t specify cycle numbers, Uproot will pick the last for you, which is almost always what you want. (In other words, you can ignore them.)

Just asking for the [uproot.TTree](https://uproot.readthedocs.io/en/latest/uproot.behaviors.TTree.TTree.html) object and printing it out _does not_ read the whole dataset.

In [ ]:
tree = file["Events"]
tree.show()

<br><br><br><br>

### Reading part of a TTree
In the last lesson, we learned that the most direct way to read one TBranch is to call [uproot.TBranch.array](https://uproot.readthedocs.io/en/latest/uproot.behaviors.TBranch.TBranch.html#array).

In [ ]:
tree["nMuon"].array()

However, it takes a long time because a lot of data have to be sent over the network.

To limit the amount of data read, set `entry_start` and `entry_stop` to the range you want. The `entry_start` is inclusive, `entry_stop` exclusive, and the first entry would be indexed by `0`, just like slices in an array interface (first lesson). Uproot only reads as many TBaskets as are needed to provide these entries.

In [ ]:
tree["nMuon"].array(entry_start=1_000, entry_stop=2_000)

These are the building blocks of a parallel data reader: each is responsible for a different slice. (See also [uproot.TTree.num_entries_for](https://uproot.readthedocs.io/en/latest/uproot.behaviors.TTree.TTree.html#num-entries-for) and [uproot.TTree.common_entry_offsets](https://uproot.readthedocs.io/en/latest/uproot.behaviors.TTree.TTree.html#common-entry-offsets), which can be used to pick `entry_start`/`entry_stop` in optimal ways.)

<br><br><br><br>

### Reading multiple TBranches at once
Suppose you know that you will need all of the muon TBranches. Asking for them in one request is more efficient than asking for each TBranch individually because the server can be working on reading the later TBaskets from disk while the earlier TBaskets are being sent over the network to you. Whereas a TBranch has an `array` method, the TTree has an `arrays` (plural) method for getting multiple arrays.

In [ ]:
muons = tree.arrays(
    ["Muon_pt", "Muon_eta", "Muon_phi", "Muon_mass", "Muon_charge"], entry_stop=1_000
)
muons

Now all five of these TBranches are in the output, `muons`, which is an Awkward Array. An Awkward Array of multiple TBranches has a dict-like interface, so we can get each variable from it by

In [ ]:
muons["Muon_pt"]
muons["Muon_eta"]
muons["Muon_phi"]  # etc.

#### Beware! It’s `tree.arrays` that actually reads the data!
If you’re not careful with the [uproot.TTree.arrays](https://uproot.readthedocs.io/en/latest/uproot.behaviors.TTree.TTree.html#arrays) call, you could end up waiting a long time for data you don’t want or you could run out of memory. Reading everything with

In [ ]:
everything = tree.arrays()

and then picking out the arrays you want is usually not a good idea. At the very least, set an `entry_stop`.

<br><br><br><br>

### Selecting TBranches by name
Suppose you have many muon TBranches and you don’t want to list them all. The [uproot.TTree.keys](https://uproot.readthedocs.io/en/latest/uproot.behaviors.TTree.TTree.html#keys) and [uproot.TTree.arrays](https://uproot.readthedocs.io/en/latest/uproot.behaviors.TTree.TTree.html#arrays) both take a `filter_name` argument that can select them in various ways (see documentation). In particular, it’s good to use the `keys` first, to know which branches match your filter, followed by `arrays`, to actually read them.

In [ ]:
tree.keys(filter_name="Muon_*")

tree.arrays(filter_name="Muon_*", entry_stop=1_000)

(There are also `filter_typename` and `filter_branch` for more options.)

<br><br><br><br>

### Scaling up, making a plot
The best way to figure out what you’re doing is to tinker with small datasets, and then scale them up. Here, we take 1000 events and compute dimuon masses.

In [ ]:
muons = tree.arrays(entry_stop=1_000)
cut = muons["nMuon"] == 2

pt0 = muons["Muon_pt", cut, 0]
pt1 = muons["Muon_pt", cut, 1]
eta0 = muons["Muon_eta", cut, 0]
eta1 = muons["Muon_eta", cut, 1]
phi0 = muons["Muon_phi", cut, 0]
phi1 = muons["Muon_phi", cut, 1]

import numpy as np

mass = np.sqrt(2 * pt0 * pt1 * (np.cosh(eta0 - eta1) - np.cos(phi0 - phi1)))

import hist

masshist = hist.Hist(hist.axis.Regular(120, 0, 120, label="mass [GeV]"))
masshist.fill(mass)
masshist.plot()

That worked (there’s a Z peak). Now to do this over the whole file, we should be more careful about what we’re reading,

In [ ]:
tree.keys(filter_name=["nMuon", "/Muon_(pt|eta|phi)/"])

and accumulate data gradually with [uproot.TTree.iterate](https://uproot.readthedocs.io/en/latest/uproot.behaviors.TTree.TTree.html#iterate). This handles the `entry_start`/`entry_stop` in a loop.

In [ ]:
masshist = hist.Hist(hist.axis.Regular(120, 0, 120, label="mass [GeV]"))

for muons, report in tree.iterate(filter_name=["nMuon", "/Muon_(pt|eta|phi)/"], report=True):
    print(f"Processing entries {report.start} to {report.stop}")
    cut = muons["nMuon"] == 2
    pt0 = muons["Muon_pt", cut, 0]
    pt1 = muons["Muon_pt", cut, 1]
    eta0 = muons["Muon_eta", cut, 0]
    eta1 = muons["Muon_eta", cut, 1]
    phi0 = muons["Muon_phi", cut, 0]
    phi1 = muons["Muon_phi", cut, 1]
    mass = np.sqrt(2 * pt0 * pt1 * (np.cosh(eta0 - eta1) - np.cos(phi0 - phi1)))
    masshist.fill(mass)
    print(masshist.sum() / tree.num_entries)

masshist.plot();

<br><br><br><br>





### Getting data into NumPy or Pandas
In all of the above examples, the `array`, `arrays`, and `iterate` methods return Awkward Arrays. The Awkward Array library is useful for exactly this kind of data (jagged arrays: more in the next lesson), but you might be working with libraries that only recognize NumPy arrays or Pandas DataFrames.

Use `library="np"` or `library="pd"` to get NumPy or Pandas, respectively.

In [ ]:
tree["nMuon"].array(library="np", entry_stop=10_000)

tree.arrays(library="np", entry_stop=10_000)

tree.arrays(library="pd", entry_stop=10_000)

NumPy is great for non-jagged data like the `"nMuon"` branch, but it has to represent an unknown number of muons per event as an array of NumPy arrays (i.e. Python objects).

Pandas can be made to represent multiple particles per event by putting this structure in a [pd.MultiIndex](https://pandas.pydata.org/pandas-docs/stable/user_guide/advanced.html), but not when the DataFrame contains more than one particle type (e.g. muons and electrons). Use separate DataFrames for these cases. If it helps, note that there’s another route to DataFrames: by reading the data as an Awkward Array and calling [ak.to_pandas](https://awkward-array.org/doc/main/reference/generated/ak.to_dataframe.html) on it. (Some methods use more memory than others, and I’ve found Pandas to be unusually memory-intensive.)

Or use Awkward Arrays (next lesson).

<br><br><br>

### 📌 Key Points

 * ROOT files have a structure that enables partial reading. This is essential for large datasets.
 * Be aware of how much data you’re reading and when.
 * The Python + Jupyter + Uproot interface provides a gradual path from interactive tinkering to scaled-up workflows.


<br><br><br><br><br>

# Jagged, ragged, Awkward Arrays

## What is Awkward Array?

The previous lesson included a tricky slice:

In [ ]:
cut = muons["nMuon"] == 2

pt0 = muons["Muon_pt", cut, 0]

The three parts of `muons["Muon_pt", cut, 0]` slice

1. selects the `"Muon_pt"` field of all records in the array,
2. applies `cut`, a boolean array, to select only events with two muons,
3. selects the first (`0`) muon from each of those pairs. Similarly for the second (`1`) muons.

NumPy would not be able to perform such a slice, or even represent an array of variable-length lists without resorting to arrays of objects.

In [ ]:
import numpy as np

# generates a ValueError
np.array([[0.0, 1.1, 2.2], [], [3.3, 4.4], [5.5], [6.6, 7.7, 8.8, 9.9]])

Awkward Array is intended to fill this gap:

In [ ]:
import awkward as ak

ak.Array([[0.0, 1.1, 2.2], [], [3.3, 4.4], [5.5], [6.6, 7.7, 8.8, 9.9]])

Arrays like this are sometimes called "[jagged arrays](https://en.wikipedia.org/wiki/Jagged_array)" and sometimes "ragged arrays."

<br><br><br><br><br>



## Slicing in Awkward Array

Basic slices are a generalization of NumPy's—what NumPy would do if it had variable-length lists.

In [ ]:
array = ak.Array([[0.0, 1.1, 2.2], [], [3.3, 4.4], [5.5], [6.6, 7.7, 8.8, 9.9]])
array.tolist()

array[2]
array[-1, 1]
array[2:, 0]
array[2:, 1:]
array[:, 0]

**Quick quiz:** why does the last one raise an error?

In [ ]:
###%cat hint1.txt

Boolean and integer slices work, too:

In [ ]:
array[[True, False, True, False, True]]

array[[2, 3, 3, 1]]

Like NumPy, boolean arrays for slices can be computed, and functions like [ak.num](https://awkward-array.readthedocs.io/en/latest/_auto/ak.num.html) are helpful for that.

In [ ]:
ak.num(array)

ak.num(array) > 0

array[ak.num(array) > 0, 0]
array[ak.num(array) > 1, 1]

Now consider this (similar to an example from the first lesson):

In [ ]:
cut = array * 10 % 2 == 0

array[cut]

This array, `cut`, is not just an array of booleans. It's a jagged array of booleans. All of its nested lists fit into `array`'s nested lists, so it can deeply select numbers, rather than selecting lists.

<br><br><br><br>

## Application: selecting particles, rather than events

Returning to the big TTree from the previous lesson,

In [ ]:
import uproot
import awkward as ak

file = uproot.open(
    ###"root://eospublic.cern.ch//eos/opendata/cms/derived-data/AOD2NanoAODOutreachTool/Run2012BC_DoubleMuParked_Muons.root"
    "data/HiggsZZ4mu.root"
)
tree = file["Events"]

muon_pt = tree["Muon_pt"].array(entry_stop=10)

This jagged array of booleans selects all *muons* with at least 20 GeV:

In [ ]:
particle_cut = muon_pt > 20

muon_pt[particle_cut]

and this non-jagged array of booleans (made with [ak.any](https://awkward-array.readthedocs.io/en/latest/_auto/ak.any.html)) selects all events *that have* a muon with at least 20 GeV:

In [ ]:
event_cut = ak.any(muon_pt > 20, axis=1)

muon_pt[event_cut]

**Quick quiz:** construct exactly the same `event_cut` using [ak.max](https://awkward-array.readthedocs.io/en/latest/_auto/ak.max.html).

**Quick quiz:** apply both cuts; that is, select muons with over 20 GeV from events that have them.

_Hint:_ you'll want to make a

In [ ]:
cleaned = muon_pt[particle_cut]

intermediary and you can't use the variable `event_cut`, as-is.


_Hint:_ the final result should be a jagged array, just like muon_pt, but with fewer lists and fewer items in those lists.

## Solution (no peeking!)

In [ ]:
cleaned = muon_pt[particle_cut]
final_result = cleaned[event_cut]
final_result.tolist()

<br><br><br><br><br>



# Combinatorics in Awkward Array

Variable-length lists present more problems than just slicing and computing formulas array-at-a-time. Often, we want to combine particles in all possible pairs (within each event) to look for decay chains.

## Pairs from two arrays, pairs from a single array

Awkward Array has functions that generate these combinations. For instance, [ak.cartesian](https://awkward-array.readthedocs.io/en/latest/_auto/ak.cartesian.html) takes a Cartesian product per event (when `axis=1`, the default).

<img src="img/cartoon-cartesian.svg" style="width: 300px">

In [ ]:
numbers = ak.Array([[1, 2, 3], [], [5, 7], [11]])
letters = ak.Array([["a", "b"], ["c"], ["d"], ["e", "f"]])

pairs = ak.cartesian((numbers, letters))

These `pairs` are 2-tuples, which are like records in how they're sliced out of an array: using strings.

In [ ]:
pairs["0"]
pairs["1"]

There's also [ak.unzip](https://awkward-array.readthedocs.io/en/latest/_auto/ak.unzip.html), which extracts every field into a separate array (opposite of [ak.zip](https://awkward-array.readthedocs.io/en/latest/_auto/ak.zip.html)).

In [ ]:
lefts, rights = ak.unzip(pairs)
lefts
rights

Note that these `lefts` and `rights` are not the original `numbers` and `letters`: they have been duplicated and have the same shape.

The Cartesian product is equivalent to this C++ `for` loop over two collections:

```cpp
for (int i = 0; i < numbers.size(); i++) {
  for (int j = 0; j < letters.size(); j++) {
    // compute formula with numbers[i] and letters[j]
  }
}
```

Sometimes, though, we want to find all pairs within a single collection, without repetition. That would be equivalent to this C++ `for` loop:

```cpp
for (int i = 0; i < numbers.size(); i++) {
  for (int j = i + 1; i < numbers.size(); j++) {
    // compute formula with numbers[i] and numbers[j]
  }
}
```

The Awkward function for this case is [ak.combinations](https://awkward-array.readthedocs.io/en/latest/_auto/ak.combinations.html).

<img src="img/cartoon-combinations.svg" style="width: 300px">

In [ ]:
pairs = ak.combinations(numbers, 2)
pairs

lefts, rights = ak.unzip(pairs)

lefts * rights  # they line up, so we can compute formulas

<br><br><br><br><br>

## Application to dimuons

The dimuon search in the previous lesson was a little naive in that we required *exactly two* muons to exist in every event and only computed the mass of that combination. If a third muon were present because it's a complex electroweak decay or because something was mismeasured, we would be blind to the other two muons. They might be real dimuons.

A better procedure would be to look for all pairs of muons in an event and apply some criteria for selecting them.

In this example, we'll [ak.zip](https://awkward-array.readthedocs.io/en/latest/_auto/ak.zip.html) the muon variables together into records.

In [ ]:
import uproot
import awkward as ak

file = uproot.open(
    ###"root://eospublic.cern.ch//eos/opendata/cms/derived-data/AOD2NanoAODOutreachTool/Run2012BC_DoubleMuParked_Muons.root"
    "data/HiggsZZ4mu.root"
)
tree = file["Events"]

arrays = tree.arrays(filter_name="/Muon_(pt|eta|phi|charge)/", entry_stop=10000)

muons = ak.zip(
    {
        "pt": arrays["Muon_pt"],
        "eta": arrays["Muon_eta"],
        "phi": arrays["Muon_phi"],
        "charge": arrays["Muon_charge"],
    }
)

arrays.type
muons.type

The difference between `arrays` and `muons` is that `arrays` contains separate lists of `"Muon_pt"`, `"Muon_eta"`, `"Muon_phi"`, `"Muon_charge"`, while `muons` contains lists of records with `"pt"`, `"eta"`, `"phi"`, `"charge"` fields.

Now we can compute pairs of muon *objects*

In [ ]:
pairs = ak.combinations(muons, 2)

pairs.type

and separate them into arrays of the first muon and the second muon in each pair.

In [ ]:
mu1, mu2 = ak.unzip(pairs)

**Quick quiz:** how would you ensure that all lists of records in `mu1` and `mu2` have the same lengths? _Hint:_ see [ak.num](https://awkward-array.readthedocs.io/en/latest/_auto/ak.num.html) and [ak.all](https://awkward-array.readthedocs.io/en/latest/_auto/ak.all.html).

Since they do have the same lengths, we can use them in a formula.

In [ ]:
import numpy as np

mass = np.sqrt(
    2 * mu1.pt * mu2.pt * (np.cosh(mu1.eta - mu2.eta) - np.cos(mu1.phi - mu2.phi))
)

**Quick quiz:** how many masses do we have in each event? How does this compare with `muons`, `mu1`, and `mu2`?

<br><br><br><br><br>

## Plotting the jagged array

Since this `mass` is a jagged array, it can't be directly histogrammed. Histograms take a set of *numbers* as inputs, but this array contains *lists*.

Supposing you just want to plot the numbers from the lists, you can use [ak.flatten](https://awkward-array.readthedocs.io/en/latest/_auto/ak.flatten.html) to flatten one level of list or [ak.ravel](https://awkward-array.readthedocs.io/en/latest/_auto/ak.ravel.html) to flatten all levels.

In [ ]:
import hist

hist.Hist(hist.axis.Regular(120, 0, 120, label="mass [GeV]")).fill(
    ak.ravel(mass)
).plot()

Alternatively, suppose you want to plot the *maximum* mass-candidate in each event, biasing it toward Z bosons? [ak.max](https://awkward-array.readthedocs.io/en/latest/_auto/ak.max.html) is a different function that picks one element from each list, when used with `axis=1`.

In [ ]:
ak.max(mass, axis=1)

Some values are `None` because there is no maximum of an empty list. [ak.flatten](https://awkward-array.readthedocs.io/en/latest/_auto/ak.flatten.html)/[ak.ravel](https://awkward-array.readthedocs.io/en/latest/_auto/ak.ravel.html) remove missing values (`None`) as well as squashing lists,

In [ ]:
ak.flatten(ak.max(mass, axis=1), axis=0)

but so does removing the empty lists in the first place.

In [ ]:
ak.max(mass[ak.num(mass) > 0], axis=1)

<br><br><br><br><br>



## Exercise: select pairs of muons with opposite charges

This is neither an event-level cut nor a particle-level cut, it is a cut on particle *pairs*.

### Solution (no peeking!)

The `mu1` and `mu2` variables are the left and right halves of muon pairs. Therefore,

In [ ]:
cut = (mu1.charge != mu2.charge)

has the right multiplicity to be applied to the `mass` array.

In [ ]:
hist.Hist(hist.axis.Regular(120, 0, 120, label="mass [GeV]")).fill(
  ak.ravel(mass[cut])
).plot()

plots the cleaned muon pairs.

<br><br><br><br><br>

## Exercise (harder): plot the one mass candidate per event that is strictly closest to the Z mass

Instead of just taking the maximum mass in each event, find the one with the minimum difference between computed mass and `zmass = 91`.

Hint: use [ak.argmin](https://awkward-array.readthedocs.io/en/latest/_auto/ak.argmin.html) with `keepdims=True`.

Anticipating one of the future lessons, you could get a more accurate mass by asking the Particle library:

In [ ]:
import particle, hepunits

zmass = particle.Particle.findall("Z0")[0].mass / hepunits.GeV

### Solution (no peeking!)

Instead of maximizing `mass`, we want to minimize `abs(mass - zmass)` and apply that choice to `mass`. [ak.argmin](https://awkward-array.readthedocs.io/en/latest/_auto/ak.argmin.html) returns the *index position* of this minimum difference, which we can then apply to the original `mass`. However, without `keepdims=True`, [ak.argmin](https://awkward-array.readthedocs.io/en/latest/_auto/ak.argmin.html) removes the dimension we would need for this array to have the same nested shape as `mass`. Therefore, we `keepdims=True` and then use [ak.ravel](https://awkward-array.readthedocs.io/en/latest/_auto/ak.ravel.html) to get rid of missing values and flatten lists.

The last step would require two applications of [ak.flatten](https://awkward-array.readthedocs.io/en/latest/_auto/ak.flatten.html): one for squashing lists at the first level and another for removing `None` at the second level.

In [ ]:
which = ak.argmin(abs(mass - zmass), axis=1, keepdims=True)
hist.Hist(hist.axis.Regular(120, 0, 120, label="mass [GeV]")).fill(
    ak.drop_none(ak.ravel(mass[which]))
).plot()

<br><br><br><br>

### 📌 Key Points

 * NumPy (and almost all array libraries) is only for rectilinear collections of numbers: arrays, tables, and tensors.
 * Awkward Array extends NumPy’s slicing and array-manipulation to jagged arrays and more general data types (such as nested records).
 * These extensions are useful for physics.
 * There’s usually more than one way to get what you want.

<br><br><br><br><br>

# Histogram manipulations and fitting

## Histogram libraries

Mainstream Python has libraries for filling histograms.

## NumPy

NumPy, for instance, has an [np.histogram](https://numpy.org/doc/stable/reference/generated/numpy.histogram.html) function.

In [ ]:
import uproot

tree = uproot.open("data/Zmumu.root")["events"]

import numpy as np

np.histogram(tree["M"].array())

Because of NumPy's prominence, this 2-tuple of arrays (bin contents and edges) is a widely recognized histogram format, though it lacks many of the features high-energy physicists expect (under/overflow, axis labels, uncertainties, etc.).

<br><br><br><br><br>

## Matplotlib

Matplotlib also has a [plt.hist](https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.hist.html) function.

In [ ]:
import matplotlib.pyplot as plt

plt.hist(tree["M"].array());

In addition to the same bin contents and edges as NumPy, Matplotlib includes a plottable graphic.

<br><br><br><br><br>

## Boost-histogram and hist

The main feature that these functions lack (without some effort) is refillability. High-energy physicists usually want to fill histograms with more data than can fit in memory, which means setting bin intervals on an empty container and filling it in batches (sequentially or in parallel).

Boost-histogram is a library designed for that purpose. It is intended as an infrastructure component. You can explore its "low-level" functionality upon importing it:

In [ ]:
import boost_histogram as bh

A more user-friendly layer (with plotting, for instance) is provided by a library called "hist."

In [ ]:
import hist

h = hist.Hist(hist.axis.Regular(120, 60, 120, name="mass"))

h.fill(tree["M"].array())

h.plot();

<br><br><br><br><br>

## Universal Histogram Indexing (UHI)

There is an attempt within Scikit-HEP to standardize what array-like slices mean for a histogram. ([See documentation](https://uhi.readthedocs.io/en/latest/indexing.html).)

Naturally, integer slices should select a range of bins,

In [ ]:
h[10:110].plot();

but often you want to select bins by coordinate value

In [ ]:
# Explicit version
h[hist.loc(90) :].plot();

In [ ]:
# Short version
h[90j:].plot();

or rebin by a factor,

In [ ]:
# Explicit version
h[:: hist.rebin(2)].plot();

In [ ]:
# Short version
h[::2j].plot();

or sum over a range.

In [ ]:
# Explicit version
h[hist.loc(80) : hist.loc(100) : sum]

In [ ]:
# Short version
h[90j:100j:sum]

Things get more interesting when a histogram has multiple dimensions.

In [ ]:
import uproot
import hist
import awkward as ak

picodst = uproot.open(
    "https://pivarski-princeton.s3.amazonaws.com/pythia_ppZee_run17emb.picoDst.root:PicoDst"
)

vertexhist = hist.Hist(
    hist.axis.Regular(600, -1, 1, label="x"),
    hist.axis.Regular(600, -1, 1, label="y"),
    hist.axis.Regular(40, -200, 200, label="z"),
)

vertex_data = picodst.arrays(filter_name="*mPrimaryVertex[XYZ]")

vertexhist.fill(
    ak.flatten(vertex_data["Event.mPrimaryVertexX"]),
    ak.flatten(vertex_data["Event.mPrimaryVertexY"]),
    ak.flatten(vertex_data["Event.mPrimaryVertexZ"]),
)

vertexhist[:, :, sum].plot2d_full();

In [ ]:
vertexhist[-0.25j:0.25j, -0.25j:0.25j, sum].plot2d_full();

In [ ]:
vertexhist[sum, sum, :].plot();

In [ ]:
vertexhist[-0.25j:0.25j:sum, -0.25j:0.25j:sum, :].plot();

A histogram object can have more dimensions than you can reasonably visualize—you can slice, rebin, and project it into something visual later.

<br><br><br><br>

# Fitting histograms

By directly writing a loss function in Minuit:

In [ ]:
import numpy as np
import iminuit.cost

norm = len(h.axes[0].widths) / (h.axes[0].edges[-1] - h.axes[0].edges[0]) / h.sum()


def f(x, background, mu, gamma):
    return (
        background
        + (1 - background) * gamma**2 / ((x - mu) ** 2 + gamma**2) / np.pi / gamma
    )


loss = iminuit.cost.LeastSquares(
    h.axes[0].centers, h.values() * norm, np.sqrt(h.variances()) * norm, f
)
loss.mask = h.variances() > 0

minimizer = iminuit.Minuit(loss, background=0, mu=91, gamma=4)

minimizer.migrad()
minimizer.hesse()

(h * norm).plot()
plt.plot(loss.x, f(loss.x, *minimizer.values));

Or through zfit, a Pythonic RooFit-like fitter:

In [ ]:
import zfit

binned_data = zfit.data.BinnedData.from_hist(h)

binning = zfit.binned.RegularBinning(120, 60, 120, name="mass")
space = zfit.Space("mass", binning=binning)

background = zfit.Parameter("background", 0)
mu = zfit.Parameter("mu", 91)
gamma = zfit.Parameter("gamma", 4)
unbinned_model = zfit.pdf.SumPDF(
    [zfit.pdf.Uniform(60, 120, space), zfit.pdf.Cauchy(mu, gamma, space)], [background]
)

model = zfit.pdf.BinnedFromUnbinnedPDF(unbinned_model, space)
loss = zfit.loss.BinnedNLL(model, binned_data)

minimizer = zfit.minimize.Minuit()
result = minimizer.minimize(loss)

binned_data.to_hist().plot(density=1)
model.to_hist().plot(density=1)

<br><br><br>

### 📌 Key Points

 * High-energy physicists approach histogramming in a different way from NumPy, Matplotlib, SciPy, etc.
 * Scikit-HEP tools make histogramming and fitting Pythonic.

<br><br><br><br><br>

# Lorentz vectors

In keeping with the "many small packages" philosophy, 2D/3D/Lorentz vectors are handled by a package named Vector. This is where you can find calculations like `deltaR` and coordinate transformations.

In [ ]:
import vector

one = vector.obj(px=1, py=0, pz=0)
two = vector.obj(px=0, py=1, pz=1)

one + two

one.deltaR(two)

one.to_rhophieta()
two.to_rhophieta()

To fit in with the rest of the ecosystem, Vector must be an array-oriented library. Arrays of 2D/3D/Lorentz vectors are processed in bulk.

`MomentumNumpy2D`, `MomentumNumpy3D`, `MomentumNumpy4D` are NumPy array subtypes: NumPy arrays can be *cast* to these types and get all the vector functions.

In [ ]:
import uproot
import awkward as ak
import vector

tree = uproot.open("data/Zmumu.root")["events"]

one = ak.to_numpy(tree.arrays(filter_name=["E1", "p[xyz]1"]))
two = ak.to_numpy(tree.arrays(filter_name=["E2", "p[xyz]2"]))

one.dtype.names = ("E", "px", "py", "pz")
two.dtype.names = ("E", "px", "py", "pz")

one = one.view(vector.MomentumNumpy4D)
two = two.view(vector.MomentumNumpy4D)

one + two

one.deltaR(two)

one.to_rhophieta()
two.to_rhophieta()

After `vector.register_awkward()` is called, `"Momentum2D"`, `"Momentum3D"`, `"Momentum4D"` are record names that Awkward Array will recognize to get all the vector functions.

In [ ]:
vector.register_awkward()

tree = uproot.open("data/uproot-HZZ.root")["events"]

array = tree.arrays(filter_name=["Muon_E", "Muon_P[xyz]"])

muons = ak.zip(
    {"px": array.Muon_Px, "py": array.Muon_Py, "pz": array.Muon_Pz, "E": array.Muon_E},
    with_name="Momentum4D",
)
mu1, mu2 = ak.unzip(ak.combinations(muons, 2))

mu1 + mu2

mu1.deltaR(mu2)

muons.to_rhophieta()

<br><br><br><br><br>

# Particle properties and PDG identifiers

The Particle library provides all of the particle masses, decay widths and more from the PDG.
It further contains a series of tools to programmatically query particle properties and use several identification schemes.


In [ ]:
import particle
from hepunits import GeV

particle.Particle.findall("pi")

z_boson = particle.Particle.from_name("Z0")
z_boson.mass / GeV, z_boson.width / GeV

print(z_boson.describe())

particle.Particle.from_pdgid(111)

particle.Particle.findall(
    lambda p: p.pdgid.is_meson and p.pdgid.has_strange and p.pdgid.has_charm
)

print(particle.PDGID(211).info())

pdgid = particle.Corsika7ID(11).to_pdgid()
particle.Particle.from_pdgid(pdgid)

<br><br><br><br><br>

# Jet clustering

In a high-energy pp collision, for instance, a spray of hadrons is produced which is clustered into `jets' of particles and this method/process is called jet-clustering.  The anti-kt jet clustering algorithm is one such algorithm used to combine particles/hadrons that are close to each other into jets.

Some people need to do jet-clustering at the analysis level. The fastjet package makes it possible to do that an (Awkward) array at a time.

In [ ]:
%pip install fastjet

In [ ]:
import uproot
import awkward as ak
import particle
from hepunits import GeV
import vector

vector.register_awkward()

picodst = uproot.open(
    "https://pivarski-princeton.s3.amazonaws.com/pythia_ppZee_run17emb.picoDst.root:PicoDst"
)
px, py, pz = ak.unzip(
    picodst.arrays(filter_name=["Track/Track.mPMomentum[XYZ]"], entry_stop=100)
)

probable_mass = particle.Particle.from_name("pi+").mass / GeV

pseudojets = ak.zip(
    {"px": px, "py": py, "pz": pz, "mass": probable_mass}, with_name="Momentum4D"
)
good_pseudojets = pseudojets[pseudojets.pt > 0.1]

import fastjet

jetdef = fastjet.JetDefinition(fastjet.antikt_algorithm, 1.0)

clusterseq = fastjet.ClusterSequence(good_pseudojets, jetdef)
clusterseq.inclusive_jets()

ak.num(good_pseudojets), ak.num(clusterseq.inclusive_jets())

This fastjet package uses Vector to get coordinate transformations and all the Lorentz vector methods you're accustomed to having in pseudo-jet objects. I used Particle to impute the mass of particles with only track-level information.

See how all the pieces accumulate?

### 📌 Key Points

 * Instead of building vector methods into multiple packages, a standalone package provides just that.
 * The value of these small packages amplify when used together.


<br><br><br><br>


## Exercise 1: Explore the content of the following file `data/Zmumu.root`:

In [ ]:
zmumu_file = uproot.open("data/Zmumu.root")
zmumu_file.classnames()

Often, the best thing to do first with an unfamiliar TTree is [show](https://uproot.readthedocs.io/en/latest/uproot.behaviors.TTree.TTree.html#show).

In [ ]:
zmumu = zmumu_file["events"]
zmumu.show()

Keep in mind that

   * TTrees are dict-like objects with subscripting (square brackets), [keys](https://uproot.readthedocs.io/en/latest/uproot.behaviors.TTree.TTree.html#keys), [values](https://uproot.readthedocs.io/en/latest/uproot.behaviors.TTree.TTree.html#values), [items](https://uproot.readthedocs.io/en/latest/uproot.behaviors.TTree.TTree.html#items), and [typenames](https://uproot.readthedocs.io/en/latest/uproot.behaviors.TTree.TTree.html#typenames) methods (like directories)
   * you can access all of the above data with methods: you don't have to parse the [show](https://uproot.readthedocs.io/en/latest/uproot.behaviors.TTree.TTree.html#show) string!

In [ ]:
zmumu.keys()

In [ ]:
zmumu.typenames()

In [ ]:
{name: branch.interpretation for name, branch in zmumu.items()}

<br><br><br><br><br>

## Exercise 2: Reading TBranches into arrays

The basic method is to get a TBranch (with square brackets) and call [array](https://uproot.readthedocs.io/en/latest/uproot.behaviors.TBranch.TBranch.html#array).

In [ ]:
zmumu["M"].array()

Some important parameters:

   * `entry_start`, `entry_stop` to limit how much you read (if it's big)
   * `library="np"` for NumPy arrays, `library="ak"` for Awkward Arrays, and `library="pd"` for Pandas (Series or DataFrame)

In [ ]:
zmumu["M"].array(entry_stop=5)

Get a NumPy (not Awkward) array:

In [ ]:
zmumu["M"].array(library="np")

Get a Pandas Series:

In [ ]:
zmumu["M"].array(library="pd")

<br><br><br><br><br>

## Exercise 3: Reading many arrays at once

The [arrays](https://uproot.readthedocs.io/en/latest/uproot.behaviors.TBranch.HasBranches.html#arrays) method retrieves many TBranches into a "group" of arrays.

What a "group" means depends on the library.

Awkward Arrays group data in records (substructure within the array).

In [ ]:
zmumu.arrays()

A "group" of NumPy arrays is a dict (unless you specify `how`):

In [ ]:
zmumu.arrays(library="np")

A "group" of Pandas Series is a DataFrame:

In [ ]:
zmumu.arrays(library="pd")

The first argument can be used to extract TBranches by name:

In [ ]:
zmumu.arrays(["px1", "py1", "px2", "py2"], library="pd")

But this argument actually takes arbitrary (Python) expressions.

In [ ]:
zmumu.arrays(["sqrt(px1**2 + py1**2)", "sqrt(px2**2 + py2**2)"], library="pd")

This is to support any [aliases](https://uproot.readthedocs.io/en/latest/uproot.behaviors.TTree.TTree.html#aliases) that might be in the [TTree](https://uproot.readthedocs.io/en/latest/uproot.behaviors.TTree.TTree.html), but you can make up your own `aliases` on the spot.

In [ ]:
zmumu.arrays(
    ["pt1", "pt2"],
    {"pt1": "sqrt(px1**2 + py1**2)", "pt2": "sqrt(px2**2 + py2**2)"},
    library="pd",
)

The fact that these are interpreted as expressions has some "gotchas":

   * nested branches, paths with "`/`", _would be interpreted as division!_
   * wildcards, paths with "`*`", _would be interpreted as multiplication!_

For pattern-matching on TBranches, use the `filter_name`, `filter_typename`, and `filter_branch` arguments of [arrays](https://uproot.readthedocs.io/en/latest/uproot.behaviors.TBranch.HasBranches.html#arrays).

In [ ]:
zmumu.arrays(filter_name="p[xyz]*", library="pd")

These filters have the same meaning as in [keys](https://uproot.readthedocs.io/en/latest/uproot.behaviors.TBranch.HasBranches.html#keys) and [typenames](https://uproot.readthedocs.io/en/latest/uproot.behaviors.TBranch.HasBranches.html#typenames), so you can test your filters without reading data.

In [ ]:
zmumu.keys(filter_name="p[xyz]*")

In [ ]:
zmumu.typenames(filter_name="p[xyz]*")

<br><br><br>

## Exercise 4: Get arrays in manageable chunks

The [iterate](https://uproot.readthedocs.io/en/latest/uproot.behaviors.TBranch.HasBranches.html#iterate) method is like [arrays](https://uproot.readthedocs.io/en/latest/uproot.behaviors.TBranch.HasBranches.html#arrays), but it can be used in a loop over chunks of the array.

How large are the chunks? You should set that with `step_size`.

In [ ]:
for arrays in zmumu.iterate(step_size=300):
    print(repr(arrays))

In [ ]:
for arrays in zmumu.iterate(step_size="50 kB"):   # 50 kB is very small! for illustrative purposes only!
    print(repr(arrays))

<br><br><br>

## Exercise 5: Collections of files (like TChain)

Each of the single-TTree array-reading methods has a corresponding multi-file function.

   * The equivalent of TTree [arrays](https://uproot.readthedocs.io/en/latest/uproot.behaviors.TTree.TTree.html#arrays) is [uproot.concatenate](https://uproot.readthedocs.io/en/latest/uproot.behaviors.TBranch.concatenate.html). _(Reads everything at once: use this as a convenience on datasets you know are small!)_
   * The equivalent of TTree [iterate](https://uproot.readthedocs.io/en/latest/uproot.behaviors.TTree.TTree.html#iterate) is [uproot.iterate](https://uproot.readthedocs.io/en/latest/uproot.behaviors.TBranch.iterate.html). _(This is the most useful one.)_

Demo of scanning through a large (remote) file:

In [ ]:
import IPython
import matplotlib.pyplot as plt
import matplotlib.pylab

In [ ]:
h = hist.Hist.new.Reg(100, 0, 500, name="mass").Double()

for muons in uproot.iterate(
    # filename(s)
    ["data/HiggsZZ4mu.root:Events"],

    # expressions
    ["pt", "eta", "phi", "charge"],
    aliases={"pt": "Muon_pt", "eta": "Muon_eta", "phi": "Muon_phi", "charge": "Muon_charge"},    

    # the all-important step_size!
    step_size="1 MB",
):
    # do everything you're going to do to this array
    cut = (ak.num(muons.charge) >= 2) & (ak.sum(muons.charge[:, :2], axis=1) == 0)
    mu1 = muons[cut, 0]
    mu2 = muons[cut, 1]

    # such as filling a histogram
    h.fill(np.sqrt(2*mu1.pt*mu2.pt*(np.cosh(mu1.eta - mu2.eta) - np.cos(mu1.phi - mu2.phi))))

    h.plot()
    plt.yscale("log")
    IPython.display.display(matplotlib.pylab.gcf())
    IPython.display.clear_output(wait=True)

    if h.counts().sum() > 300000:
        break

<br><br><br><br><br>

## Exercise 6: What if the data are not one-dimensional or rectilinear arrays?

Consider this ROOT file (simulation used in the Higgs discovery, converted to NanoAOD).

In [ ]:
events = uproot.open("data/HiggsZZ4mu.root:Events")

Some data, such as missing energy (MET), consist of a single value per collision event and can be represented by normal NumPy arrays.

In [ ]:
nonjagged = events["MET_pt"].array(entry_stop=20, library="np")
nonjagged

Normal NumPy slicing rules apply.

In [ ]:
nonjagged[:5]

<br><br><br><br><br>

Some data cannot. There's a different number of muons in each event, so we need variable-length nested lists to represent their transverse momenta ($p_T$).

In [ ]:
jagged_awkward = events["Muon_pt"].array(entry_stop=20)
jagged_awkward

In [ ]:
jagged_awkward.tolist()

It is _possible_ to read these data into NumPy, but with a considerable cost.

In [ ]:
jagged_numpy = events["Muon_pt"].array(entry_stop=20, library="np")
jagged_numpy

This is an _array of NumPy arrays_ (because you can put Python objects in NumPy arrays—not recommended).

You might want to consider the inner arrays to be a second dimension and use normal slicing rules:

In [ ]:
jagged_awkward[:, :1]

In [ ]:
jagged_numpy[:, :1]

In NumPy, you can't. NumPy doesn't know that all the contents of this array are arrays of the same type.

You know this and can write a loop in Python:

In [ ]:
np.array([x[:1] for x in jagged_numpy], dtype=object)

But this isn't recommended. It's non-idiomatic and slow.

<br><br><br><br><br>

Pandas can work with this "jagged" data through indexing.

In [ ]:
events.arrays(filter_name=["Muon_*"], library="pd")

But there are limitations. Try loading non-muon branches in the same DataFrame.

<br><br><br><br><br>

## Awkward Array

Awkward Array is a library for manipulating JSON-like data using NumPy-like idioms.

<img src="img/cartoon-schematic.svg" width="800px"/>

<br><br><br><br><br>

The documentation is at [https://awkward-array.org/](https://awkward-array.org/).

<img src="img/awkward-documentation.png" width="800px"/>

<br><br><br><br><br>

Consider this Parquet file of the same dataset. It can be read into an Awkward Array, just like the ROOT file.

In [ ]:
array = ak.from_parquet("data/HiggsZZ4mu.parquet")
array

In [ ]:
array.fields

In [ ]:
array[0].tolist()

In [ ]:
array.muons.pt

<br><br><br><br><br>

I've restructured it a little ("NanoEvents-style" vs "NanoAOD-style"), but we can easily make the data from ROOT look just like the data from Parquet.

In [ ]:
nanoaod_style = events.arrays(filter_name="Muon_*")
nanoaod_style.type

In [ ]:
array.muons.type

In [ ]:
nanoevents_style = ak.zip({
    "pt": nanoaod_style.Muon_pt,
    "eta": nanoaod_style.Muon_eta,
    "phi": nanoaod_style.Muon_phi,
    "mass": nanoaod_style.Muon_mass,
    "charge": nanoaod_style.Muon_charge,
})
nanoevents_style.type

<br><br><br><br><br>

In general, Awkward Array data types "`T`" can be:

   * numbers, booleans, date-times, etc.
   * variable-length and fixed-length lists of `T`
   * records with named or unnamed (tuple) fields of type `T1`, `T2`, ...
   * missing values: `T` _or_ `None`
   * heterogeneous types: `T1` _or_ `T2` _or_ ...

<br><br><br><br><br>

## Changing structure

Although the `tolist()` form of the data looks like JSON objects, the data are actually in a very fluid "columnar" form.

You can, for instance, turn an array from entry-per-event into entry-per-lumiblock by increasing the nesting by one.

<img src="img/events-to-lumis.svg" width="800px"/>

In [ ]:
array.luminosityBlock

In [ ]:
lumilengths = ak.run_lengths(array.luminosityBlock)
lumilengths

In [ ]:
array_by_lumi = ak.unflatten(array, lumilengths, axis=0)
array_by_lumi

This is an array of lists (luminosity blocks) of records (collision events) containing lists of records (muons, generator-level particles, etc.).

In [ ]:
array_by_lumi.luminosityBlock[0]

In [ ]:
array.type

In [ ]:
array_by_lumi.type

So, for example, you can add up $p_T$ values along any of three dimensions.

In [ ]:
ak.sum(array_by_lumi.muons.pt, axis=-1)

Summing (or "reducing" in general) is well-defined for irregular data shapes, depending on a choice of alignment. We left-align.

<img src="img/example-reduction.svg" style="width: 800px">

<br><br><br><br><br>

## Practical analysis

To get beyond theory, let's do some realistic things with the data.

In [ ]:
array.muons

A cut is an array of booleans.

In [ ]:
cut = ak.num(array.muons) >= 2
cut

In [ ]:
array.muons[cut]

Notice that there are now fewer events than there had been before.

Sometimes, that's a problem for composing cuts—having to know which cuts have been applied to which arrays of booleans.

We can avoid that problem by masking: rejected events are replaced by "`None`", rather than removed.

In [ ]:
m = array.muons
m.mask[cut]
m

In [ ]:
selected_muons = array.muons[cut]

selected_muons.charge[:, 0] + selected_muons.charge[:, 1] == 0

The `[:, 0]`, `[:, 1]` syntax assumes that a first and second muon exists (it does, because of the selection) and ignores all others.

In you want to consider combinations of all good particles in each event, so there are functions for constructing that.

<table style="margin-left: 0px">
    <tr style="background: white"><td style="font-size: 1.75em; font-weight: bold; text-align: center"><a href="https://awkward-array.readthedocs.io/en/latest/_auto/ak.cartesian.html">ak.cartesian</a></td><td style="font-size: 1.75em; font-weight: bold; text-align: center"><a href="https://awkward-array.readthedocs.io/en/latest/_auto/ak.combinations.html">ak.combinations</a></td></tr>
    <tr style="background: white"><td><img src="img/cartoon-cartesian.svg"></td><td><img src="img/cartoon-combinations.svg"></td></tr>
</table>

In [ ]:
ak.combinations(array.muons, 2).type

In [ ]:
mu1, mu2 = ak.unzip(ak.combinations(array.muons, 2))
mu1, mu2

These are not just the first two muons in each event: they are all combinations of two (without duplication).

In [ ]:
h = hist.Hist.new.Reg(120, 0, 120, name="mass").Double()
h.fill(ak.flatten(
    np.sqrt(2*mu1.pt*mu2.pt*(np.cosh(mu1.eta - mu2.eta) - np.cos(mu1.phi - mu2.phi)))
))

<br><br><br><br><br>

## Example: reconstructed/generator-level matching with ΔR

In [ ]:
gen = ak.with_name(array.gen, "Momentum3D")
gen

In [ ]:
gen.fields

First, make all reco-gen pairs. (`nested=True` puts all pairs with a given reco muon in a new nested list.)

In [ ]:
reco_gen = ak.cartesian({"muon": muons, "gen": gen}, nested=True)
reco_gen

Break them into two arrays to make them easier to work with.

In [ ]:
mu, g = ak.unzip(reco_gen)
mu, g

Now we can compute an array of ΔR values.

In [ ]:
mu.deltaR(g)

In [ ]:
hist.Hist.new.Reg(100, 0, 5).Double().fill(
    ak.flatten(mu.deltaR(g), axis=None)
)

Some are very close to zero, some not.

How about if we look only at generator-level muons? Only non-muons?

In [ ]:
hist.Hist.new.Reg(100, 0, 5).Double().fill(
    ak.flatten(mu.deltaR(g)[abs(g.pdgId) == 13], axis=None)
)

What we want is not ΔR for _all_ reco-gen pairs, but the minimum ΔR for each reco muon.

[ak.min](https://awkward-array.readthedocs.io/en/latest/_auto/ak.min.html) is a reducer, removing that extra layer of nested list we made with `nested=True`.

In [ ]:
mu.deltaR(g)

In [ ]:
ak.min(mu.deltaR(g), axis=-1)

Zoom into small ranges of ΔR.

In [ ]:
hist.Hist.new.Reg(100, 0, 5).Double().fill(
    ak.flatten(ak.min(mu.deltaR(g), axis=-1), axis=None)
)

Instead of just plotting the minimum, let's get the [ak.argmin](https://awkward-array.readthedocs.io/en/latest/_auto/ak.argmin.html), the index position of the best ΔR.

(`keepdims=True` keeps the reducer from removing a dimension, which we'll need for the slice in the next step. The best indexes are in length-1 lists.)

In [ ]:
best = ak.argmin(mu.deltaR(g), axis=-1, keepdims=True)
best

This slice picks out the reco-gen pairs with minimal ΔR.

In [ ]:
reco_gen[best]

And there you have it: an array of `{muon: ABC, gen: XYZ}` pairs representing the best match for each reco muon.

In [ ]:
ak.flatten(reco_gen[best], axis=-1)[:4].tolist()

<br><br><br><br><br>

## Numba: a just-in-time compiler for Python

It's possible to do complex combinatorics with array-at-a-time functions, but nested "for" loops would often be easier.

Nested "for" loops can be fast if they're compiled.

[Numba](https://numba.pydata.org/) compiles Python.

In [ ]:
import numba as nb

Remember how long it takes to run a loop in Python?

In [ ]:
starttime = time.time()

sumpt = np.zeros(len(array), np.float64)
for i, event in enumerate(array):
    for muon in event.muons:
        sumpt[i] += muon.pt

python_time = time.time() - starttime
print(f"total time: {python_time} sec")

The same loop, in a function preceded by `@nb.jit`, is compiled by Numba when you first call it.

In [ ]:
@nb.jit
def calculate_sumpt(array):
    out = np.zeros(len(array), np.float64)
    for i, event in enumerate(array):
        for muon in event.muons:
            out[i] += muon.pt
    return out

In [ ]:
calculate_sumpt(array)

In [ ]:
starttime = time.time()

sumpt = calculate_sumpt(array)

numba_time = time.time() - starttime
print(f"total time: {numba_time} sec")

In [ ]:
python_time / numba_time

In many cases, Numba is _faster_ than the corresponding array-at-a-time function.

In [ ]:
starttime = time.time()

sumpt = ak.sum(array.muons.pt, axis=-1)

awkward_time = time.time() - starttime
print(f"total time: {awkward_time} sec")

In [ ]:
python_time / awkward_time

**Conclusion:** use array-at-a-time functions when you're working interactively or it's the most concise/easy-to-understand way to write an expression.

Use Numba when you need extreme speed or "for" loops are the most concise/easy-to-understand way to write it.

Convoluted code, just for the sake of using array-at-a-time functions, is not helping anybody!

<br><br><br><br><br>

### Limitations of Numba

Maybe this sounds too good to be true: "Python is slow, but put `@nb.jit` on each function and it will be fast."

The truth is that Numba only works on a _subset_ of Python. It replaces Python code with statically typed, compiled code, and Python is too dynamic of a language for that to always be possible. The Numba team keeps a list of [supported Python language features](https://numba.readthedocs.io/en/stable/reference/pysupported.html) and [supported NumPy functions](https://numba.readthedocs.io/en/stable/reference/numpysupported.html). Numba also only recognizes libraries that have been explicitly extended to work with it. Awkward Array and Vector have been extended; hist will be.

When it fails, the error messages can be hard to understand. Hint: start with a small, do-nothing function and gradually fold in the features you want, to better know which part is causing the error.

If you're willing to learn a new language, [Julia](https://julialang.org/) is designed from the ground up as a just-in-time compilable language.



<br><br><br><br><br>

## Reconstructed/generator-level matching in Numba

Before we repeat the reco-gen matching exercise in Numba, let's build a simple Awkward Array output from a Numba-compiled function using [ak.ArrayBuilder](https://awkward-array.readthedocs.io/en/latest/_auto/ak.ArrayBuilder.html).

In [ ]:
@nb.jit
def build_nested(array, builder):
    for event in array:
        builder.begin_list()
        
        for muon in event.muons:
            builder.append(muon.pt)
        
        builder.end_list()
    
    return builder

build_nested(array, ak.ArrayBuilder()).snapshot()

It's the same as `array.muons.pt`, so this is definitely an example where the array-at-a-time function is simpler.

In [ ]:
array.muons.pt

Reco-gen matching, however, is simpler as a nested "for" loop.

Note that we don't have to output the fully formed array; it is enough to use Numba to make the index that we slice arrays with outside of the Numba-compiled function.

In [ ]:
@nb.jit
def matching(array_muons, array_gen, builder):
    for muons_event, gen_event in zip(array_muons, array_gen):
        builder.begin_list()

        for muon in muons_event:
            best_i = -1
            best_dr = -1.0
            for i, gen in enumerate(gen_event):
                dr = muon.deltaR(gen)
                if best_i < 0 or dr < best_dr:
                    best_i = i
                    best_dr = dr

            if best_i < 0:
                builder.append(None)
            else:
                builder.append(best_i)

        builder.end_list()

    return builder

index_of_best = matching(muons, gen, ak.ArrayBuilder()).snapshot()
index_of_best

This index picks the best generator-level particle for each reconstructed muon.

In [ ]:
gen_match = gen[index_of_best]
gen_match

In [ ]:
ak.num(gen), ak.num(muons), ak.num(gen_match)

So building the reco-gen pairs is just an [ak.zip](https://awkward-array.readthedocs.io/en/latest/_auto/ak.zip.html).

_This_ part would be harder in Numba. Use the best tool for each job.

In [ ]:
ak.zip({"muons": muons, "gen": gen_match})

<br><br><br><br><br>

## Example: identifying Z bosons in H → ZZ → 4μ

The Higgs boson decays to an "on-shell" Z boson (with a mass near 91 GeV) and an "off-shell" Z boson (much lower mass).

In a real analysis, it is necessary to know which is which, because different quality cuts are applied. Given only the four muons, finding the right pair of pairs is a combinatorics problem.

This example solves that problem using only array-at-a-time functions.

In [ ]:
four_muons = muons[(ak.num(muons) == 4) & (ak.sum(muons.charge, axis=-1) == 0)]
four_muons

General strategy: identify qualitatively distinct collections as separate named arrays.

The names will help you in thinking about the problem.

In [ ]:
mu_plus = four_muons[four_muons.charge > 0]
mu_minus = four_muons[four_muons.charge < 0]
mu_plus, mu_minus

By construction (the cut defining `four_muons`), all lists in `mu_plus` and `mu_minus` have exactly two items each.

In [ ]:
ak.num(mu_plus), ak.num(mu_minus)

You can check that explicitly to increase confidence and (if necessary) debug.

In [ ]:
ak.all(ak.num(mu_plus) == 2), ak.all(ak.num(mu_minus) == 2)

Knowing this (and the fact that 2 is not a large number), we can name each of these to further simplify our structures.

In [ ]:
mu_plus_0 = mu_plus[:, 0]
mu_plus_1 = mu_plus[:, 1]
mu_minus_0 = mu_minus[:, 0]
mu_minus_1 = mu_minus[:, 1]

mu_plus_0, mu_plus_1, mu_minus_0, mu_minus_1

Now, we _could_ do combinatorics using [ak.combinations](https://awkward-array.readthedocs.io/en/latest/_auto/ak.combinations.html), but with such a small number of known combinations, do it with explicitly named arrays. The structures will be simpler and the names will help you.

For each event, either `z00` and `z11` will be valid or `z01` and `z10` will be valid. The names help you remember this constraint to avoid using the same particle in multiple decays.

In [ ]:
z00 = mu_plus_0 + mu_minus_0
z11 = mu_plus_1 + mu_minus_1

z01 = mu_plus_0 + mu_minus_1
z10 = mu_plus_1 + mu_minus_0

As an aside, instead of hard-coding the Z mass, get it from the [particle](https://github.com/scikit-hep/particle#readme) package, which is like a Pythonic PDG.

In [ ]:
import particle

In [ ]:
particle.Particle.from_name("Z0")

In [ ]:
particle.Particle.from_name("Z0").mass

In [ ]:
zGeV = particle.Particle.from_name("Z0").mass / 1000

Another aside, [np.minimum](https://numpy.org/doc/stable/reference/generated/numpy.minimum.html) is an array-at-a-time function to find element-by-element minima between two arrays.

In [ ]:
np.minimum(np.array([1, 2, 3, 4, 5]), np.array([5, 4, 3, 2, 1]))

We can use that to find the closest-to-on-shell Z in each of the two scenarios: 0011 (`z00` and `z11`) or 0110 (`z01` and `z10`).

In [ ]:
zdist_0011 = np.minimum(abs(z00.mass - zGeV), abs(z11.mass - zGeV))
zdist_0110 = np.minimum(abs(z01.mass - zGeV), abs(z10.mass - zGeV))
zdist_0011, zdist_0110

For each event `i`, either `zdist_0011[i]` is nearly zero because it contains a real on-shell Z and a real off-shell Z or `zdist_0110[i]` is.

For each `i`, the wrong case has two crossed muon pairs, not correctly identified Zs, which are both far from 91 GeV.

In [ ]:
is_0011 = zdist_0011 < zdist_0110
is_0011

With an array of booleans, we can pick the right pairing element-by-element using [ak.where](https://awkward-array.readthedocs.io/en/latest/_auto/ak.where.html).

Try negating the booleans with `~`.

In [ ]:
hist.Hist.new.Reg(100, 0, 100, name="distance from 91 GeV").Double().fill(
    ak.where(is_0011, zdist_0011, zdist_0110)
)

Now we'd like to select _objects_ with `is_0011`, rather than numbers, so we first have to build the right objects.

Each element needs to be two Z bosons, an array of length-2 lists of Lorentz vectors.

We can construct this individually for the array of 0011 cases and the array of 0110 cases.

`np.newaxis` unflattens an array by putting each element in a length-1 nested list. (See the [slicing documentation](https://awkward-array.readthedocs.io/en/latest/_auto/ak.Array.html#ak-array-getitem).)

In [ ]:
z00.type

In [ ]:
z00[:, np.newaxis].type

And we can [ak.concatenate](https://awkward-array.readthedocs.io/en/latest/_auto/ak.concatenate.html) two such things at `axis=1` to get length-2 lists.

In [ ]:
z0011_pair = ak.concatenate((z00[:, np.newaxis], z11[:, np.newaxis]), axis=1)
z0110_pair = ak.concatenate((z01[:, np.newaxis], z10[:, np.newaxis]), axis=1)
z0011_pair, z0110_pair

In [ ]:
ak.num(z0011_pair), ak.num(z0110_pair)

Now apply [ak.where](https://awkward-array.readthedocs.io/en/latest/_auto/ak.where.html) list by list to pick the right pair of Zs for each event.

In [ ]:
correct_pair = ak.where(is_0011[..., None], z0011_pair, z0110_pair)
correct_pair

In [ ]:
ak.num(correct_pair)

In [ ]:
hist.Hist.new.Reg(120, 0, 120, name="mass").Double().fill(
    correct_pair[:, 0].mass
)

For completeness, look at the wrong pairs:

In [ ]:
hist.Hist.new.Reg(120, 0, 120, name="mass").Double().fill(
    ak.where(~is_0011[..., None], z0011_pair, z0110_pair)[:, 0].mass
)

Each of these length-2 lists has a correctly reconstructed on-shell Z and a correctly reconstructed off-shell Z, but in no particular order.

We can sort them by their distance to 91 GeV.

In [ ]:
sort_index = ak.argsort(abs(correct_pair.mass - zGeV), axis=1)
sort_index

In [ ]:
sorted_pair = correct_pair[sort_index]
sorted_pair

In [ ]:
hist.Hist.new.Reg(120, 0, 120, name="mass").Double().fill(
    sorted_pair[:, 0].mass
)

Adding the Lorentz vectors of both Zs gives us the Higgs mass (though we didn't have to correctly identify them to do this):

In [ ]:
hist.Hist.new.Reg(150, 0, 150, name="mass").Double().fill(
    (sorted_pair[:, 0] + sorted_pair[:, 1]).mass
)

<br><br><br><br><br>

## Identifying Z bosons in H → ZZ → 4μ with Numba

Now let's do the same thing with Numba, starting with the hard part: pairing muons into right and wrong combinations and using proximity to 91 GeV of _one_ member of each pair of Z candidates to identify the right combination.

In [ ]:
four_muons

In [ ]:
ak.num(four_muons)

In [ ]:
ak.sum(four_muons.charge, axis=-1)

Pre-sorting the muons with an array-at-a-time function makes it easier to write the Numba-compiled function.

In [ ]:
sorted_four_muons = four_muons[ak.argsort(four_muons.charge, axis=1)]
sorted_four_muons.charge

The logic for picking the right pair is similar to the array-at-a-time case, but instead of "`z00[i]`" being a single Z candidate with `z00` being an array over all events, `z00` is a single Z candidate in a for loop over events.

In this example, we use the [ak.ArrayBuilder](https://awkward-array.readthedocs.io/en/latest/_auto/ak.ArrayBuilder.html) to make the output vector records directly. Just be sure to name them `"Momentum4D"`, so that they are recognized as vectors and not generic records.

(TODO: it would be nice if ArrayBuilder's `append` would take Lorentz vector objects without having to rebuild them...)

In [ ]:
@nb.jit
def make_z_pairs(sorted_four_muons, builder):
    for event in sorted_four_muons:
        # unpack the sorted four muons into appropriately named variables
        mum0, mum1, mup0, mup1 = event

        # either z00 and z11 are correct or z01 and z10 are correct
        z00 = mup0 + mum0
        z11 = mup1 + mum1
        
        z01 = mup0 + mum1
        z10 = mup1 + mum0
        
        # the correct pair of Zs is the pair that has one Z close to 91 GeV
        zdist_0011 = min(abs(z00.mass - zGeV), abs(z11.mass - zGeV))
        zdist_0110 = min(abs(z01.mass - zGeV), abs(z10.mass - zGeV))

        if zdist_0011 < zdist_0110:
            # z00 and z11 are correct
            builder.begin_list()
            builder.begin_record("Momentum4D")
            builder.field("px"); builder.append(z00.px)
            builder.field("py"); builder.append(z00.py)
            builder.field("pz"); builder.append(z00.pz)
            builder.field("E"); builder.append(z00.E)
            builder.end_record()
            builder.begin_record("Momentum4D")
            builder.field("px"); builder.append(z11.px)
            builder.field("py"); builder.append(z11.py)
            builder.field("pz"); builder.append(z11.pz)
            builder.field("E"); builder.append(z11.E)
            builder.end_record()
            builder.end_list()

        else:
            # z01 and z10 are correct
            builder.begin_list()
            builder.begin_record("Momentum4D")
            builder.field("px"); builder.append(z01.px)
            builder.field("py"); builder.append(z01.py)
            builder.field("pz"); builder.append(z01.pz)
            builder.field("E"); builder.append(z01.E)
            builder.end_record()
            builder.begin_record("Momentum4D")
            builder.field("px"); builder.append(z10.px)
            builder.field("py"); builder.append(z10.py)
            builder.field("pz"); builder.append(z10.pz)
            builder.field("E"); builder.append(z10.E)
            builder.end_record()
            builder.end_list()

    return builder

correct_pair = make_z_pairs(sorted_four_muons, ak.ArrayBuilder()).snapshot()
correct_pair

In [ ]:
hist.Hist.new.Reg(120, 0, 120, name="mass").Double().fill(
    correct_pair[:, 0].mass
)

Try inverting the selection to make the `correct_pair` contain the incorrect pair!

In [ ]:
hist.Hist.new.Reg(150, 0, 150, name="mass").Double().fill(
    (correct_pair[:, 0] + correct_pair[:, 1]).mass
)